# AdventureWorks – Försäljningsanalys

Denna notebook innehåller en analys av försäljningsdata från AdventureWorks-databasen.
Data hämtas via SQL och analyseras vidare i Python med Pandas och visualiseras med Matplotlib.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from sqlalchemy import create_engine, text
from urllib.parse import quote_plus

server = "localhost,1433"
database = "AdventureWorks2025"
username = "sa"
password = quote_plus("Str0ngPass!2025")
driver = quote_plus("ODBC Driver 18 for SQL Server")

connection_string = (
    f"mssql+pyodbc://{username}:{password}@{server}/{database}"
    f"?driver={driver}&Encrypt=yes&TrustServerCertificate=yes"
)

engine = create_engine(connection_string)

In [ ]:
with engine.connect() as conn:
    print(conn.execute(text("SELECT DB_NAME()")).scalar())

In [ ]:
def read_sql_df(query: str) -> pd.DataFrame:
    with engine.connect() as conn:
        return pd.read_sql(text(query), conn)

## 1. Antal produkter per kategori

In [ ]:
query_1 = """
SELECT
    pc.Name AS Category,
    COUNT(DISTINCT p.ProductID) AS ProductCount
FROM Production.ProductCategory pc
JOIN Production.ProductSubcategory psc
    ON psc.ProductCategoryID = pc.ProductCategoryID
JOIN Production.Product p
    ON p.ProductSubcategoryID = psc.ProductSubcategoryID
GROUP BY pc.Name
ORDER BY ProductCount DESC;
"""

df_1 = read_sql_df(query_1)
df_1

In [ ]:
plt.figure()
plt.bar(df_1["Category"], df_1["ProductCount"])
plt.title("Antal produkter per kategori")
plt.xlabel("Kategori")
plt.ylabel("Antal produkter")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()


In [ ]:
most = df_1.iloc[0]
least = df_1.iloc[-1]
most["Category"], int(most["ProductCount"]), least["Category"], int(least["ProductCount"])


**Insikt:** Kategorien med flest produkter är **Bikes** med **99** produkter.
Kategorien med minst produkter är **Clothing** med **25** produkter.

## 2. Försäljning per produktkategori

In [ ]:
query_2 = """
SELECT
    pc.Name AS Category,
    SUM(sod.LineTotal) AS TotalSales
FROM Sales.SalesOrderDetail sod
JOIN Production.Product p
    ON sod.ProductID = p.ProductID
JOIN Production.ProductSubcategory psc
    ON p.ProductSubcategoryID = psc.ProductSubcategoryID
JOIN Production.ProductCategory pc
    ON psc.ProductCategoryID = pc.ProductCategoryID
GROUP BY pc.Name
ORDER BY TotalSales DESC;
"""

df_2 = read_sql_df(query_2)
df_2

In [ ]:
df_2_plot = df_2.sort_values("TotalSales", ascending=True)

plt.figure()
plt.barh(df_2_plot["Category"], df_2_plot["TotalSales"])
plt.title("Total försäljning per produktkategori")
plt.xlabel("Total försäljning")
plt.ylabel("Kategori")
plt.tight_layout()
plt.show()

In [ ]:
highest = df_2.iloc[0]
lowest = df_2.iloc[-1]

highest["Category"], round(highest["TotalSales"], 2), lowest["Category"], round(lowest["TotalSales"], 2)

**Insikt:** Den kategori som genererat högst total försäljning är **"Bikes"**.
Den kategori som genererat lägst total försäljning är **"Accessories"**.

## 3. Försäljningstrend per månad

In [ ]:
query_3 = """
SELECT
    DATEFROMPARTS(YEAR(OrderDate), MONTH(OrderDate), 1) AS MonthStart,
    SUM(TotalDue) AS TotalSales
FROM Sales.SalesOrderHeader
GROUP BY DATEFROMPARTS(YEAR(OrderDate), MONTH(OrderDate), 1)
ORDER BY MonthStart;
"""

df_3 = read_sql_df(query_3)
df_3["MonthStart"] = pd.to_datetime(df_3["MonthStart"])
df_3

In [ ]:
plt.figure()
plt.plot(df_3["MonthStart"], df_3["TotalSales"])
plt.title("Försäljningstrend per månad")
plt.xlabel("Månad")
plt.ylabel("Total försäljning")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

In [ ]:
highest_month = df_3.loc[df_3["TotalSales"].idxmax()]
lowest_month = df_3.loc[df_3["TotalSales"].idxmin()]

highest_month["MonthStart"].strftime("%Y-%m"), round(float(highest_month["TotalSales"]), 2), \
lowest_month["MonthStart"].strftime("%Y-%m"), round(float(lowest_month["TotalSales"]), 2)


**Insikt:** Försäljningen över tid varierar tydligt mellan olika månader.
Den högsta försäljningen sker under **2025-04** med **5 847 164,69**, medan den lägsta försäljningen sker under **2025-06** med **52 478,19**.

## 4. Antal ordrar och total försäljning per år


In [ ]:
query_4 = """
SELECT
    YEAR(OrderDate) AS OrderYear,
    COUNT(SalesOrderID) AS OrderCount,
    SUM(TotalDue) AS TotalSales
FROM Sales.SalesOrderHeader
GROUP BY YEAR(OrderDate)
ORDER BY OrderYear;
"""

df_4 = read_sql_df(query_4)
df_4


In [ ]:
plt.figure()

x = range(len(df_4))

plt.bar(x, df_4["OrderCount"], label="Antal ordrar")
plt.bar(x, df_4["TotalSales"], label="Total försäljning")

plt.xticks(x, df_4["OrderYear"])
plt.title("Antal ordrar och total försäljning per år")
plt.xlabel("År")
plt.ylabel("Värde")
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
most_orders = df_4.loc[df_4["OrderCount"].idxmax()]
most_sales = df_4.loc[df_4["TotalSales"].idxmax()]

most_orders["OrderYear"], int(most_orders["OrderCount"]), \
most_sales["OrderYear"], round(float(most_sales["TotalSales"]), 2)


**Insikt:** Flest ordrar lades under **2024** med **14 244** ordrar.
Den högsta totala försäljningen uppnåddes också under **2024** med **49 020 486,51**.

## 5. Topp 10 produkter baserat på försäljning


In [ ]:
query_5 = """
SELECT TOP 10
    p.Name AS ProductName,
    SUM(sod.LineTotal) AS TotalSales
FROM Sales.SalesOrderDetail sod
JOIN Production.Product p
    ON sod.ProductID = p.ProductID
GROUP BY p.Name
ORDER BY TotalSales DESC;
"""

df_5 = read_sql_df(query_5)
df_5


In [ ]:
df_5_plot = df_5.sort_values("TotalSales", ascending=True)

plt.figure()
plt.barh(df_5_plot["ProductName"], df_5_plot["TotalSales"])
plt.title("Topp 10 produkter baserat på försäljning")
plt.xlabel("Total försäljning")
plt.ylabel("Produkt")
plt.tight_layout()
plt.show()


In [ ]:
top_product = df_5.iloc[0]

top_product["ProductName"], round(float(top_product["TotalSales"]), 2)


**Insikt:** Den produkt som genererat högst total försäljning är **Mountain-200 Black** med **4400592.8** i försäljning.

## 6. Försäljning och antal kunder per region

In [ ]:
query_6 = """
SELECT
    cr.Name AS Region,
    COUNT(DISTINCT soh.CustomerID) AS CustomerCount,
    SUM(soh.TotalDue) AS TotalSales
FROM Sales.SalesOrderHeader soh
JOIN Sales.SalesTerritory st
    ON soh.TerritoryID = st.TerritoryID
JOIN Person.CountryRegion cr
    ON st.CountryRegionCode = cr.CountryRegionCode
GROUP BY cr.Name
ORDER BY TotalSales DESC;
"""

df_6 = read_sql_df(query_6)
df_6


In [ ]:
df_6_plot = df_6.sort_values("TotalSales", ascending=True)

plt.figure()
plt.barh(df_6_plot["Region"], df_6_plot["TotalSales"])
plt.title("Total försäljning per region")
plt.xlabel("Total försäljning")
plt.ylabel("Region")
plt.tight_layout()
plt.show()


In [ ]:
top_region = df_6.iloc[0]
bottom_region = df_6.iloc[-1]

top_region["Region"], round(float(top_region["TotalSales"]), 2), \
bottom_region["Region"], round(float(bottom_region["TotalSales"]), 2)


**Insikt:** Regionen med högst total försäljning är **United States** med **70829863.2**.
Regionen med lägst total försäljning är **Germany** med **5479819.58**.
